# Experiment
In this notebook the simple Frozen Lake environment of Gymnasium will be study.
## Imports

In [1]:
import gymnasium as gym
import numpy as np

## Setting up the Environments
I created one for training to go faster and another for visualising

In [ ]:
training_env = gym.make('FrozenLake-v1', map_name="8x8", is_slippery=True)
testing_env = gym.make('FrozenLake-v1', map_name="8x8", is_slippery=True, render_mode="human")

This are the states and actions

In [3]:
states = training_env.observation_space
actions = training_env.action_space

## Collection the Data Points

In [4]:
STEPS = 500000
current_state, _ = training_env.reset()
num_actions = actions.n
num_states = states.n
gamma = 0.99

data_points = np.zeros((STEPS, 4))


for i in range(STEPS):
    action = np.random.choice(range(num_actions))
    next_state, reward, terminated, truncated, info = training_env.step(action)
    data_points[i] = [current_state, action, reward, next_state]
    current_state = next_state
    if terminated or truncated:
        current_state, info = training_env.reset()

## Building the Reward Matrix and Transition Tensor

In [5]:
transition_tensor = np.zeros((states.n, actions.n, states.n), dtype=np.float32)

state_action_counter = np.zeros((states.n, actions.n), dtype=np.int32)
# for each action states x states
state_action_tensor = np.zeros((states.n, actions.n,states.n), dtype=np.float32)

reward_matrix = np.zeros((states.n, actions.n), dtype=np.float32)

for i in range(STEPS):
    current_state, action, reward, next_state = data_points[i]
    current_state = int(current_state)
    action = int(action)
    next_state = int(next_state)
    state_action_counter[current_state, action] += 1
    state_action_tensor[current_state, action, next_state] += 1
    reward_matrix[current_state, action] += reward

# Normalize transition_tensor over next states (axis=2), not over actions!
# transition_tensor[state, action, next_state] = P(next_state | state, action)
transition_tensor = state_action_tensor / np.sum(state_action_tensor, axis=2, keepdims=True)
# Normalize reward_matrix by the count of (state, action) visits
reward_matrix = reward_matrix / np.maximum(state_action_counter, 1)  # Avoid division by zero

/tmp/ipykernel_189485/4150191650.py:20: RuntimeWarning: invalid value encountered in divide
  transition_tensor = state_action_tensor / np.sum(state_action_tensor, axis=2, keepdims=True)


Just filtering some NaNs

In [6]:
# turn all the nans into 0
reward_matrix = np.nan_to_num(reward_matrix, -1)

transition_tensor = np.nan_to_num(transition_tensor)

array([[[0.66608393, 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.3304396 , 0.33450422, 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.33200338, 0.3398833 , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.6666332 , 0.3333668 , 0.        , ..., 0.        ,
         0.        , 0.        ]],

       [[0.34360543, 0.33278254, 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.32882285, 0.        , 0.33546406, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.34055266, 0.33615467, ..., 0.        ,
         0.        , 0.        ],
        [0.33227047, 0.3364402 , 0.33128935, ..., 0.        ,
         0.        , 0.        ]],

       [[0.        , 0.3314394 , 0.34212664, ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.3295228 , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.3

## The core Value Iteration algorithm
Here a simple [[Iteration Algorithms#Value Iteration|Value Iteration]] was implemented, which can be further optimised to build the transition tensor, and reward through Value Iteration. 

In [7]:
err = 1
i = 0

V = np.ones(states.n)

while err > 1e-8:
  Q = np.zeros((states.n, num_actions))

  for action in range(num_actions):
    Q[:,action] = reward_matrix[:,action] + gamma * transition_tensor[:,action] @ V

  Vnew = np.max(Q, axis=1)
  err = np.linalg.norm(V - Vnew)
  V = Vnew
  i+=1

np.argmax(Q, axis=1)

array([3, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 2, 2, 1, 3, 3, 0, 0, 2, 3,
       2, 2, 3, 3, 3, 1, 0, 0, 2, 2, 0, 3, 3, 0, 2, 1, 3, 2, 0, 0, 0, 1,
       3, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 2, 0, 1, 0, 0, 1, 0, 1, 0])

## Testing

In [11]:
import time

obs, info = testing_env.reset()

while True:
    action = np.argmax(Q[obs])
    obs, reward, terminated, truncated, info = testing_env.step(action)
    # sleep for 0.1 seconds
    time.sleep(0.1)
    if terminated or truncated:
        break

error: display Surface quit

Note that sometimes it will fall into the whole since it has a probability of falling on it. 

In [9]:
testing_env.close()